In [39]:
from g4f.client import Client
from db_utils import csv_read,query_database
import re

In [40]:
client = Client()

In [41]:
def generate_sql_from_nl(user_input):
    csv_string = csv_read()
    system_prompt = f'''
        1. 你是一個財務資料庫專家，能夠生成相應的SQL指令。 
        2. SQL的編寫請用全英文，結束請記得給分號。
        3. 非常重要!返回sql語法即可，不用其他說明與文字。
        4. MYSQL code中請不要出現資料庫中沒有的英文名稱。
        5. 請務必生成完整SQL語法，並一定要在首欄加入年或季度的欄位，例如seasonid。
       '''

    user_prompt = f'''
        1.資料庫結構、表名稱、欄位名稱請務必參考以下內容:{csv_string}
        2.一定不能生成沒有的sql表名與欄名，欄位名稱請盡量比對中文對照!
        3.請務必要判斷是否需要跨表查詢，如需要請使用join指令。
        4.另外時間只有年與季度seasonsID，例如2010年第一季格式為201001、2010年第二季格式為201002、
        2020年第三季格式為202003、2020年第四季格式為202004
                   '''
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input + user_prompt}
        ],
        max_tokens=150,
        temperature=0.1
    )
    sql_response = response.choices[0].message.content
    print(f"回答:{sql_response}")
    return sql_response


模型可用系列:GPT、Gemini

測試生成sql，gemini pro生成結果比較精簡，速度快。gpt4omini比較能判斷複雜sql欄位。

In [42]:
def extract_sql_queries(sql_response):
    matches = re.findall(r'SELECT.*?;', sql_response, re.IGNORECASE | re.DOTALL) 
    if matches:
        # 移除每個匹配項中的換行符號 
        matches = [match.replace('\n', ' ') for match in matches]
        return matches 
    else: 
        raise ValueError("SQL查詢指令未找到")

In [43]:
def classify_user_input(user_input:str):
    # 初次調用判斷是否需要function calling 
    response = client.chat.completions.create( model="gpt-4o-mini", 
                                                messages=[
                                                {"role": "system", 
                                                 "content":'''1.請判斷使用者的輸入是否需要連線到「財務報表」資料庫查詢，若需要請回答finvision即可!
                                                              2.若使用者的輸入是關於分析財務指標，請返回analysis即可!
                                                              3.若都不屬於查詢與分析範圍，只要回答NO即可!
                                                              4.財務報表資料庫內容包含資產負債表、損益表、現金流量表，其餘公司資料都沒有。'''},
                                                {"role": "user", "content": user_input}])
    
    print('===== 初次調用GPT判斷 =====')
    if 'finvision' in response.choices[0].message.content:
        sql_response = generate_sql_from_nl(user_input) 
        sql_query = extract_sql_queries(sql_response)
        print(f'生成的SQL指令: {sql_query}') 
        # 執行GPT生成的SQL查詢 
        report = query_database(sql_query)
        print(f'查詢結果: {report}')
        return(f"finvision select :{report}")

    else: 
        # 一般對話處理
        return(response.choices[0].message.content)
         


In [ ]:
def main():
    user_input ="請給我2023年各季度之利息收入、法定盈餘公積、銷貨收入"
    report = classify_user_input(user_input)
    return(report)

In [44]:
main()

===== 初次調用GPT判斷 =====
回答:```sql
SELECT 
    seasonsID, 
    II AS Interest_Income, 
    LR AS Legal_Reserve, 
    NSR AS Sales_Revenue 
FROM 
    pl 
WHERE 
    seasonsID BETWEEN 202301 AND 202304;
```
生成的SQL指令: ['SELECT      seasonsID,      II AS Interest_Income,      LR AS Legal_Reserve,      NSR AS Sales_Revenue  FROM      pl  WHERE      seasonsID BETWEEN 202301 AND 202304;']
連線成功
查詢結果: "SQL query error: (1054, \"Unknown column 'LR' in 'field list'\")"


'finvision select'